In [11]:
%matplotlib inline
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import json
import http

def companies_list():
    symb=[]
    with open('nasdaq-company-list.csv', newline='') as csvfile:
        reader=csv.DictReader(csvfile)
        for row in reader:
            symb.append(row['Symbol'])
        symb.append('MSFT')
        symb.append('APPL')
        symb.append('fb')
        symb.append('nflx')
        symb.append('fvrr')
        symb.append('sbux')
        symb.append('mcb')
        symb.append('m')
        symb.append('kirk')
        symb.append('rdsa')
        symb.append('tsla')
        return symb[:20]
    

def get_timeseries(symbol):
    conec = http.client.HTTPSConnection('www.alphavantage.co', timeout = 10)
    conec.request("GET", '/query?function=TIME_SERIES_WEEKLY&symbol='+symbol+'&interval=5min&apikey=07THEOFGYUDV073A')
    resp=conec.getresponse()
    return json.loads(resp.read().decode('utf8'))

#for trainning
from pandas_datareader import data
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import keras
import csv
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Convolution1D, Flatten, MaxPooling1D
import datetime as dt
import time 
import urllib.request, json
import os
import numpy as np
#import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from alphavantage import time_series_weekly

# variables: '1. open', '2. high', '3. low', '4. close'
# Make plots bigger
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

close=[]
test=[]
low=[]
high=[]
    #print(timestamp) -> from 2019 to begining
    #]\\print(test)

def make_timeseries_regressor(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=4):
    model = Sequential((
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        MaxPooling1D(),
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='linear'),
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model


def make_timeseries_instances(timeseries, window_size):
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0]
    X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
    y = timeseries[window_size:]
    q = np.atleast_3d([timeseries[-window_size:]])
    return X, y, q


def evaluate_timeseries(model, timeseries, window_size):
    filter_length = 5
    nb_filter = 4
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       #1D vectors -> 2D column vectors

    nb_samples, nb_series = timeseries.shape
   # model = make_timeseries_regressor(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)

    #model.summary()

    X, y, q = make_timeseries_instances(timeseries, window_size)
    test_size = int(0.2 * nb_samples)
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    model.fit(X_train, y_train, nb_epoch=5, batch_size=2, validation_data=(X_test, y_test))

    print(type(model))
    pred = model.predict(X_test)
   # print('\n\nactual', 'predicted', sep='\t')
    #for actual, predicted in zip(y_test, pred.squeeze()):
     #   print(actual.squeeze(), predicted, sep='\t')
    print('next', model.predict(q).squeeze(), sep='\t')
    return model.predict(q)


#def company()
def load_json():
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model=model_from_json(loaded_model_json)
    loaded_model.load_weights("model.h5")
    return loaded_model

def predict(loaded_model, symbol):
    print(symbol)
    t=time_series_weekly(symbol)
    inTimeseries = []
    for timestamp, data in t.items():
        clos=float(data['4. close'])
        inTimeseries.insert(0, clos)
   # print(inTimeseries)
    return loaded_model.predict(np.atleast_3d(inTimeseries))
#company_list=pd.read_csv("nasdaq-company-list.csv")
#print(company_list)

    
def main():
    """Prepare input data, build model, evaluate."""
    np.set_printoptions(threshold=25)
    companies=companies_list()
    window_size = 200
    training_threshold = window_size * 2
    json_file = open('model.json', 'r')
    model_json = json_file.read()
    json_file.close()
    model=model_from_json(model_json)
    model.load_weights("model.h5")
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    #model = make_timeseries_regressor(window_size, 4, nb_input_series=1, nb_outputs=1, nb_filter=4)
    
    for symbol in companies:
        jro = None
        try:
            jro = time_series_weekly(symbol)
        except:
            continue
        ts_length = len(jro.items())
        if ts_length < training_threshold:
            continue
        inTimeseries = []
        for timestamp, data in jro.items():
            clos=float(data['4. close'])
            inTimeseries.insert(0, clos)
        filter_length=4
        evaluate_timeseries(model, inTimeseries, window_size)
        time.sleep(15)
    modelJson = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(modelJson)
    model.save_weights("model.h5")

    print('a')
    
main()

C:\Users\Radu\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 636 samples, validate on 208 samples
Epoch 1/5
636/636 [==============================] - 3s 5ms/step - loss: 1.3340 - mean_absolute_error: 0.8765 - val_loss: 3.3782 - val_mean_absolute_error: 1.4028
Epoch 2/5
636/636 [==============================] - 2s 4ms/step - loss: 1.1533 - mean_absolute_error: 0.8110 - val_loss: 2.9249 - val_mean_absolute_error: 1.1349
Epoch 3/5
636/636 [==============================] - 2s 4ms/step - loss: 1.1140 - mean_absolute_error: 0.7855 - val_loss: 5.2950 - val_mean_absolute_error: 1.8452
Epoch 4/5
636/636 [==============================] - 2s 4ms/step - loss: 1.0265 - mean_absolute_error: 0.7562 - val_loss: 3.3697 - val_mean_absolute_error: 1.3366
Epoch 5/5
636/636 [==============================] - 2s 4ms/step - loss: 0.9611 - mean_absolute_error: 0.7327 - val_loss: 2.8771 - val_mean_absolute_error: 1.1536
<class 'keras.engine.sequential.Sequential'>
next	14.798561
Train on 636 samples, validate on 208 samples
Epoch 1/5
636/636 [==============

636/636 [==============================] - 2s 4ms/step - loss: 0.5110 - mean_absolute_error: 0.5469 - val_loss: 0.1731 - val_mean_absolute_error: 0.3410
<class 'keras.engine.sequential.Sequential'>
next	1.1516724
Train on 506 samples, validate on 176 samples
Epoch 1/5
506/506 [==============================] - 2s 4ms/step - loss: 12.0608 - mean_absolute_error: 2.5135 - val_loss: 0.7130 - val_mean_absolute_error: 0.5853
Epoch 2/5
506/506 [==============================] - 2s 4ms/step - loss: 10.4533 - mean_absolute_error: 2.3201 - val_loss: 0.8500 - val_mean_absolute_error: 0.7721
Epoch 3/5
506/506 [==============================] - 2s 4ms/step - loss: 10.0473 - mean_absolute_error: 2.3056 - val_loss: 0.7191 - val_mean_absolute_error: 0.7075
Epoch 4/5
506/506 [==============================] - 2s 4ms/step - loss: 8.5597 - mean_absolute_error: 2.1131 - val_loss: 0.5645 - val_mean_absolute_error: 0.5292
Epoch 5/5
506/506 [==============================] - 2s 4ms/step - loss: 8.7650 - mean

In [ ]:
#ts = TimeSeries(key='07THEOFGYUDV073A', output_format='pandas')

window_size=100
N=training_data.size
avg_predictions=[]
avg_x=[]
for pred_idx in range(window_size,N):

    if pred_idx >= N:
        date = jro.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        date = jro.loc[pred_idx,'']

    avg_predictions.append(np.mean(train_data[pred_idx-window_size:pred_idx]))
    mse_errors.append((avg_predictions[-1]-training_data[pred_idx])**2)
    std_avg_x.append(date)

print('MSE predic %.5f'%(0.5*np.mean(mse_errors)))






In [ ]:
#main
print("Close value")
    print('\nSimple single timeseries vector prediction')
                # The timeseries f(t) = t
    evaluate_timeseries(mid, window_size)

    print('\nMultiple-input, multiple-output prediction')
    timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
    evaluate_timeseries(mid, window_size)
    
    print("Low value")
    print('\nSimple single timeseries vector prediction')
    timeseries = np.arange(ts_length)                   # The timeseries f(t) = t
    evaluate_timeseries(low, window_size)

    print('\nMultiple-input, multiple-output prediction')
    timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
    evaluate_timeseries(low, window_size)
    
    print("High Value")
    print('\nSimple single timeseries vector prediction')
    timeseries = np.arange(ts_length)                   # The timeseries f(t) = t
    evaluate_timeseries(high, window_size)

    print('\nMultiple-input, multiple-output prediction')
    timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
    evaluate_timeseries(high, window_size)
    

In [ ]:
#EMA = 0.0
#gamma = 0.1
#for ti in range(11000):
 # EMA = gamma*train_reshape[ti] + (1-gamma)*EMA
  #train_reshape[ti] = EMA